In [6]:
#IMPORTS

import enchant  #pip install pyenchant
import sys,os
from os import path
import getpass
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import itertools
import ast
import nltk 
#pip install nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [7]:
def extractWordAfterSymbol(text,symbol):
    
    """ 
    Generalizando, como hizo mauro
    
    RECIBE: Un texto plano que puede tener cualquier cosa
    DEVUELVE: Una lista de todos los strings que estaban precedidos por un simbolo(sea @, #, etc)
    EJEMPLO: 'Hola que tal soy @axel, y vivo en @capitalFederal' -----> [axel,capitalFederal]'
    """
    
    words = text.split(' ')
    
    users = []
    userToSave = ""
    for word in words:
        if len(word) != 0 and symbol in word:
            splittedWord = word.split(symbol)
            if len(splittedWord) > 1:
                userToSave = splittedWord[1]
            else:
                userToSave = splittedWord[0]
        if userToSave != '':
            users.append(userToSave)
            userToSave = ""
    return users

In [8]:
def removeDots(word):
    """ 
    RECIBE: Una palabra cualquiera
    DEVUELVE: La parte de la palabra que no tiene ningun punto
    EJEMPLO: 'esperando...' -----> 'esperando'
    """
    
    splited = word.split('.')
    
    for chunk in splited:
        if chunk != '.':
            return chunk
    return word

In [9]:
def hasNumeric(word):
    """ 
    RECIBE: Una palabra cualquiera
    DEVUELVE: True o False dependiendo de si contiene algun numero
    EJEMPLO: 'hol4' -----> True
    """    
    
    for c in word:
        if c.isnumeric():
            return True
    return False

In [10]:
def getEnlgishWord(word):
       
    """
    RECIBE: Un texto plano que puede tener cualquier cosa
    DEVUELVE: La una tupla con la palabra mas larga en ingles que encuentre primero dentro del texto recibido
              y un True o False dependiendo de si habia o no una palabra en ingles en el texto. 
              Si no contiene ninguna palabra en ingles devuelve el texto original.
    TENER EN CUENTA: No considera como una palabra valida a las que tengan longitud 1
    EJEMPLO: 'aaaaaaaaaaENGLISHaaaaaaaaaa' -----> 'ENGLISH'
    EJEMPLO: 'aaaaaaaaaaaaaaaaaaaa' -----> 'aaaaaaaaaaaaaaaaaaaa'
    """  
    
    englishDictionary = enchant.Dict("en_US")
    
    MIN_LEN_WORDS = 2
    
    failure = False
    
    if word == None:
        failure = True
    
    elif len(word) < MIN_LEN_WORDS:
        failure = True
        
    else:
        for i in range(len(word), MIN_LEN_WORDS - 1,-1):
            for j in range(i):
                aux = word[j:i]
                if englishDictionary.check(aux) and not hasNumeric(aux) and len(aux) > 1:
                    return removeDots(aux), True
                else:
                    failure = True
    
    if failure:  
        return word, False
        

In [11]:
def mapBinaryLabel(binaryIndicator, labelsList):
    
    """ 
    RECIBE: 1 o 0
    DEVUELVE: labelsList[0] o labelsList[1] respectivamente
    """  
    
    if binaryIndicator == 1:
        return labelsList[0]
    if binaryIndicator == 0:
        return labelsList[1]
    else:
        return None

In [12]:
def generateAbsolutePath(fileName, exists = False, nombreRepo = 'DatosRepo'):
    
    """ 
    RECIBE: El nombre de un archivo
            exists es para indicar que el archivo ya existe. En cuyo caso lo va a buscar a donde este
    DEVUELVE: El path absoluto del archivo
    TENER EN CUENTA: Si se pone mal escrito el archivo es posible que lo encuentre igual por ej 'trai' en vez de 'train.csv'
                     Asume que tu repo se llama DatosRepo a menos que se lo digas en <nombreRepo>
    EJEMPLO: 'miArchivo.txt' -----> '/home/axelmpm/DatosRepo/miArchivo.txt'
    """
    
    path = '/home/' + getpass.getuser() + '/' + nombreRepo
    
    if exists:
        for dirpath, dirnames, filenames in os.walk(path):
            for filename in [f for f in filenames if (os.path.basename(f) == fileName)]:
                return os.path.join(dirpath, filename)
    else:
        return os.path.abspath("") + '/' + fileName

In [13]:
def readDic(fileName, transformStringedListToList = False):
    
    """ 
    RECIBE: El nombre de un archivo .csv
    DEVUELVE: El diccionario que estaba almacenado en disco
    """
    
    df = pd.read_csv(generateAbsolutePath(fileName + '.csv'))
    
    if transformStringedListToList:
        df['values'] = df.apply(lambda row : ast.literal_eval(row['values']), axis = 1)
    
    dic = {}
    
    keys = list(df['keys'])
    values = list(df['values'])
    
    for i in range(len(keys)):
        dic[keys[i]] = values[i]
    
    return dic

In [14]:
def getWordInListFromFile(fileName):
    
    """ 
    RECIBE: El nombre de un archivo donde cada linea tiene palabras
    DEVUELVE: Una lista donde cada elemento son las palabras que estaban como linea en el archivo (no toma las palabras de longitud 1)
    """ 
    
    file = open(generateAbsolutePath(fileName),'r')
    
    l = []
    
    for line in file:
        if len(line[:-1]) > 1:
            l.append(line[:-1]) #le saco el \n
    file.close()
    return l

In [15]:
def hasCertainWords(stuffWithWordsInIt,words):
    
    """ 
    RECIBE: Recibe algo que tiene palabras dentro (texto, lista de palabras, conjuntos con palabras, etc)
    DEVUELVE: Devuelve True o False dependiendo de si el el contendor contiene ALGUNA palabra en la lista de palabras
    """ 
    
    for word in words:
        if word in stuffWithWordsInIt:
            return True
    return False

In [16]:
def clasifyWordsRespectToEnglish(bagOfWords):
    
    """ 
    RECIBE: Un contenedor de palabras que podrian o no estar en ingles
    DEVUELVE: Una lista de palabras que estan en ingles y otra que no estaban en ingles
    EJEMPLO: {'hello..gggggg,ghghghgh,who,hola'} -------> {hello,who}, {ghghghgh,hola}
    """ 
    
    bagOfEnglishWords = []
    bagOfNonEnglishWords = []
    for word in bagOfWords:
        processedWord,isEnglish = getEnlgishWord(word)
        if isEnglish:
            bagOfEnglishWords.append(processedWord)
        else:
            bagOfNonEnglishWords.append(processedWord)
        
    return bagOfEnglishWords, bagOfNonEnglishWords

In [17]:
def getDataFrameOfFrecuencies(elementsLabel,elements, sort = True, sortAscending = False):
    
    """ 
    RECIBE: Una lista o secuencia de elementos y un nombre para estos
    DEVUELVE: Un dataframe con dos columnas, una de labels y otro de las frecuencias
    """ 
    
    temp_elements = list(elements.copy())
    frecs_dict = {i:temp_elements.count(i) for i in temp_elements}

    df = pd.DataFrame({elementsLabel : list(frecs_dict.keys()), 'Cantidad' : list(frecs_dict.values())})
    
    if sort:
        df.sort_values('Cantidad', ascending = sortAscending, inplace = True)
    return df

In [18]:
def dicToDataFrame(dic, keysLabel = "keys", valuesLabel = "values", sort = True, sortAscending = False):
    
    """ 
    RECIBE: Un diccionario y nombres para sus claves y valores y si si quiere ordenar o no ascendente o no
    DEVUELVE: Un dataframe con dos columnas, una de keys y otro de las values
    """
    
    df = pd.DataFrame({keysLabel : list(dic.keys()), valuesLabel : list(dic.values())})
    
    if sort:
        df.sort_values(valuesLabel, ascending = sortAscending, inplace = True)
    return df

In [19]:
def getHammingSimilarity(word1,word2):
    
    """ 
    RECIBE: Dos palabras de igual longitud
    DEVUELVE: Un numero del 0 al 1 donde 1 significa que son el mismo string y 0 es que no se parecen en nada
    EJEMPLO: word1 = casa, word2 = capa ---->  0.75 (porque tienen en comun la primera c, la segunda a
             y la ultima a, osea 3/4 bien)
    """     
    
    length = len(word1)
    mismatches = 0
    
    for i in range(length):
        if word1[i] != word2[i]:
            mismatches += 1
    return 1 - mismatches/length
    

In [20]:
def getAllConstantSum(k,n):

    """ 
    RECIBE: k es el valor a sumar de forma constante, n es la cantidad de numeros a sumar
    DEVUELVE: Todas las listas de n numeros que sumados dan k 
    """ 
    
    if n == 1:
        return [[k]]
    
    sums = []
    
    for i in range(k + 1):
        
        subsums = getAllConstantSum(k - i, n -1)
        
        for subsum in subsums:
            sums.append([i] + subsum)
    
    return sums

In [21]:
def generateWordExtentionsIntoNChars(word,n,extentionChar):
    
    """ 
    RECIBE: Una palabra a extender, la cantidad de caracteres que debe tener la version extendida y que caracter usar al extender
    DEVUELVE: Devuelve un set de todas las posibles extensiones
    EJEMPLO: word = 'casa', n = 6, extentionChar = '.' ---->
                {'..casa','.c.asa','.ca.sa','.cas.a','.casa.','c..asa','c.a.sa','c.as.a','c.asa.','ca..sa'
                    'ca.s.a','ca.sa.','cas..a','cas.a.','casa..'}   
    """
    
    if len(word) == '':
        return [extentionChar * n]
    
    if (len(word) > n):
        return [word]
    
    if len(word) == n:
        return [word]
    
    wordLength = len(word)
    amountOfCharsToAdd = n - wordLength
    
    extentions = set()
    
    amounts = getAllConstantSum(amountOfCharsToAdd,wordLength + 1)
    
    for amountOfExtendingChars in amounts:

        extention = ''
        for i in range(len(word)):
            extention += (extentionChar * amountOfExtendingChars[i]) + word[i]
        extention += extentionChar * amountOfExtendingChars[-1]
        
        extentions.add(''.join(list(extention)))
    
    return extentions

In [22]:
def getSimilarity(text1,text2):
    
    """ 
    RECIBE: Dos textos de cualquier longitud (no muy largos)
    DEVUELVE: Un numero del 0 al 1 donde 1 significa que son el mismo string y 0 es que no se parecen en nada
    A TENER EN CUENTA: Usa la similaridad de Hamming
                       Es mas o menos costosa a partir de 20 caracteres de texto aprox
                       Remueve los espacios por lo que no hace falta preprocesar texto
    EJEMPLO: word1 = casa, word2 = ca ---->  0.5 (porque tienen en comun la primera c, la segunda a, osea 2/4 bien)
    """ 
    
    EMPTY_CHARACTER = ' '
    
    if text1 + text2 == '':
        return 1
    
    if text1 + text2 == text1 or text1 + text2 == text2:
        return 0
    
    splitedText1 = text1.split(' ')
    splitedText2 = text2.split(' ')
    
    concatenatedWord1 = (''.join(splitedText1)).lower()
    concatenatedWord2 = (''.join(splitedText2)).lower()
    
    if len(concatenatedWord1) > len(concatenatedWord2):
        longest = concatenatedWord1
        shortest = concatenatedWord2
    else:
        longest = concatenatedWord2
        shortest = concatenatedWord1
        
    lenDif = len(longest) - len(shortest)
    maxSimilarity = 0
    extentions = generateWordExtentionsIntoNChars(shortest,len(longest),EMPTY_CHARACTER)
    
    for extention in extentions:
        
        subSimilarity = getHammingSimilarity(extention,longest)
        #print(shortest,subWord,longest,subSimilarity)
        
        if (subSimilarity >= maxSimilarity):
            maxSimilarity = subSimilarity
            
    return maxSimilarity
        

In [23]:
def getMaxSimilarityWord(word,words):

    """ 
    RECIBE: El una palabra y palabras a comparar
    DEVUELVE: La palabra que mas se parece
    """
    
    maxSimilarity = 0
    mostSimilarKnownWord = ''
    
    if word in words:
        return word
    
    candidateWords = []
    for w in words:
        if len(w) > 0 and w[0].lower() == word[0].lower():
            candidateWords.append(w)
    
    for knownWord in candidateWords:
        
        similarity = getSimilarity(word,knownWord)

        if maxSimilarity <= similarity:

            maxSimilarity = similarity
            mostSimilarKnownWord = knownWord
            
    return mostSimilarKnownWord

In [24]:
def isSimilar(text1, text2, threshold = 0.7):

    """ 
    RECIBE: Dos textos (no muy grandes)
    DEVUELVE: Si la similitud es mayor a un threshold
    """ 
    
    return getSimilarity(text1,text2) >= threshold

In [25]:
def isIn(element,container):

    """ 
    RECIBE: Un elemento y un contenedor de elementos
    DEVUELVE: Si el elemento esta o no en el contenedor
    """ 
    
    return element in container

In [26]:
def acortar_texto(df):
    """
    RECIBE: un df
    DEVUELVE: el mismo df con una columna con version corta de la columna 'text'
    """
    
    
    df['short_text'] = 'None'
    cant_filas = len(df.index)
    for y in range(cant_filas):
        df['short_text'][y:]= df.iloc[y][0][:15] + "..."

In [57]:
def hasQuestion(text):
    '''
    RECIBE: Un string.
    DEVUELVE: True si tiene al menos una palabra en ingles seguida por al menos un signo de pregunta, sino retorna FALSE
    '''
    text = text.replace('?', ' ? ')
    englishDictionary = enchant.Dict("en_US")
    text = takeOutSpecialCaractersFromText(text, specialCharacters = [',','.','#','!','%','*','(',')','"','-','_', '$', ':', '/', '\''])
    textSplitted = text.split()
    for word in textSplitted:
        if englishDictionary.check(word):
            for j in range(textSplitted.index(word) + 1, len(textSplitted)):
                if ('?' in textSplitted[j]):
                    return True
            return False
    return False

In [28]:
def takeOutSpecialCaractersFromText(text, specialCharacters = None):
    '''
    RECIBE: Un string.
    DEVUELVE: El string sin caracteres especiales. Con el parametro specialCharacters se puede decidir que caracteres sacar.
    '''
    if specialCharacters == None:
        specialCharacters = ['&','\n','<','>','=',',','.','#','!','%','*','(',')','"','-','_', '?', '$', ':', '/','\''] + [str(i) for i in range(0,10)]
    
    
    for char in specialCharacters:
        text = text.replace(char, ' ')
    return text

In [29]:
def separateOutBiaSpecialCaractersAndMayuscFromText(text, specialCharacters = None):
    '''
    RECIBE: Un string.
    DEVUELVE: Una lista de los substrings que fuerons separados por los caracteres especiales o las mayusuclas.
               Con el parametro specialCharacters se puede decidir que caracteres sacar.
    '''
    if specialCharacters == None:
        specialCharacters = ['&','\n','<','>','=',',','.','#','!','%','*','(',')',
                             '"','-','_', '?', '$', ':', '/','\''] + [str(i) for i in range(0,10)]
    
    words = []
    subWord = ''
    prevC = ''
    
    for c in text:
        
        if c.isupper() and prevC.islower():
            words.append(subWord)
            subWord = c
        else:        
            if c in specialCharacters:
                if subWord != '':
                    words.append(subWord)
                    subWord = ''
            else:
                subWord += c
        prevC = c
        
    if subWord != '':
        words.append(subWord)
        
    return words

In [30]:
def barplot(x,y,data,xLabel = None,yLabel = None,title = '',
            palette = 'Spectral',figX = 25,figY = 10,fontSize = 1.7):
    
    '''
    RECIBE:
        x: es el label de la columna con los datos de x del dataFrame 'data'
        y: es el label de la columna con los datos de y del dataFrame 'data'
        data: es el dataframe de donde sacar los datos
        xLabel: es el label para x en el plot
        yLabel: es el label para y en el plot
        xLabel: es el titulo del plot
        palette: es la paleta de colores a usar
        figX: es el tamaño en x del plot
        figY: es el tamaño en y del plot
        fontSize: es el tamaño del font en el plot
    A TENER EN CUENTA: No afecta al dataframe 'data'
    DEVUELVE: El plot
    '''
    
    if xLabel == None:
        xLabel = x
    
    if yLabel == None:
        yLabel = y
    
    sns.set(rc = {'figure.figsize':(figX,figY)})
    sns.set(font_scale = fontSize)
    v = sns.barplot(x = x, y = y, data = data, 
                 palette = palette, saturation = 1)
    
    v.set(xlabel = xLabel, ylabel= yLabel)
    v.set(title = title)
    
    return v

In [31]:
def addContextualKnowledge(wordList,context):

    """ 
    RECIBE: Una lista de palabras y un contexto de textos previos
    DEVUELVE: El contexto actualizado con el contenido del texto pasado
    """
    
    words = list(set(wordList))
    pairs = []
    
    if len(words) == 1:
        pairs.append((words[0],words[0]))
    
    else:
        for i in range(len(words)):
            for j in range(i + 1,len(words)):
                if j < len(words):
                    pairs.append((words[i],words[j]))
    
    for pair in pairs:
        
        if pair in context:
            context[pair] += 1
        else:
            context[pair] = 1
            
    return context

In [32]:
def getContextualKnowledge(texts):

    """ 
    RECIBE: Una lista de listas de palabras (lista de tweets)
    DEVUELVE: Un diccionario llamado contexto que tiene como claves combinaciones de palabras que se dieron
                juntas en algun texto de la lista y cuya clave es la cantidad de textos en los que se dio
    """
    
    context = {}
    
    for wordList in texts:
        
        context = addContextualKnowledge(wordList,context)
    
    return context

In [33]:
def getPairsWith(word,context):

    """ 
    RECIBE: Una palabra y un contexto
    DEVUELVE: Devuelve los pares de palabras del contexto que contienen a la palabra pasada
    """
    
    pairsWith = []
    
    for pair in context:
        
        if word in pair:
            pairsWith.append(pair)
    
    return pairsWith

In [34]:
def getAsociatedWordsWithReinforcement(word,context):

    """ 
    RECIBE: Una palabra y un contexto
    DEVUELVE: Una lista de (palabra,reinforcements) que seria como las palabras y la frecuencia con la que
                las vio juntos a word y a palabra
    """
    
    asociatedPairs = getPairsWith(word,context)
    asociatedWordAndReinforcement = []

    for asociatedPair in asociatedPairs:
        
        if word != asociatedPair[0]:
            asociatedWord = asociatedPair[0]
        else:
            asociatedWord = asociatedPair[1]

        asociatedWordAndReinforcement.append((asociatedWord,context[asociatedPair]))
  
    return asociatedWordAndReinforcement

In [35]:
def getContextualMap(context):
    
    """ 
    RECIBE: Un contexto
    DEVUELVE: Un diccionario que contiene como clave todas las palabras del contexto y como value sus asociadas
                con su reinforcement
    """
    
    contextualMap = {}
    
    for pair in context:
        
        for word in pair:
            
            if not word in contextualMap:

                asociatedWordAndReinforcement = getAsociatedWordsWithReinforcement(word,context)

                contextualMap[word] = asociatedWordAndReinforcement
            
    return contextualMap

In [36]:
def mergeIntoWords(texts):
    
    """ 
    RECIBE: Una lista de lista de palabras
    DEVUELVE: Una lista de las palabras de los textos recibidos
    """
    return list(set(itertools.chain.from_iterable(texts)))                
        

In [37]:
def cleanUpWords(texts):
    
    """ 
    RECIBE: Una lista de textos
    DEVUELVE: Una lista de lista de palabras (lista de tweets) cuyas palabras de los textos recibidos no son vacias
    """
    
    cleanedTexts = []
    
    stopWords =  set(stopwords.words('english'))
    
    for text in texts:
    
        words = text.split(' ')
        cleanedWords = []
        
        for word in words:
            
            subWords = separateOutBiaSpecialCaractersAndMayuscFromText(word) 
            
            for subWord in subWords:
                
                if subWord == '':
                    continue
                else:
                    cleanedWords.append(subWord.lower())
        
        cleanedTexts.append(cleanedWords)
        
    return cleanedTexts

In [38]:
def furtherCleanUp(words,stopWords):

    """ 
    RECIBE: Una lista de palabras y unas stopwords
    DEVUELVE: Una lista de palabras que no tienen longitud menor a 3 ni son stopwords
    """
    
    cleanWords = []
    
    for word in words:
        
        if not (len(word) < 3 or word in stopWords):
            cleanWords.append(word)
            
    return cleanWords

In [39]:
def truncateMapTo(contextualMap,words):
    
    """ 
    RECIBE: Toma un contextual map y unas palabras
    DEVUELVE: Un contextual map cuyas palabras en key y en los values no estan en la lista de palabras recibidas
    """
    
    truncatedContextualMap = {}
    
    for word in contextualMap:
        
        if word in words:
            asociates = contextualMap[word]

            filteredAsociates = []

            i = 0
            checksForEveryWordInAsociates = [pair[0] in words for pair in asociates]

            for asociatedWordIsIn in checksForEveryWordInAsociates:

                if asociatedWordIsIn:
                    filteredAsociates.append(asociates[i])
                i += 1

            truncatedContextualMap[word] = filteredAsociates
    return truncatedContextualMap

In [40]:
def readFromFileIntoList(fileName, strings = False):
    
    """ 
    RECIBE: Un nombre de archivo que se supone que almacena una lista previamente almacenada 
            con el metodo listToFile
    DEVUELVE: La lista guardada en ese archivo
    """
    
    file = open(generateAbsolutePath(fileName),'r')
    
    strList = (file.read()).split('&&&')[:-1]
    
    recoveredList = []
    
    for e in strList:
        
        if strings:
            recoveredList.append(e)
        else:
            recoveredList.append(ast.literal_eval(e))
        
    return recoveredList
    

In [41]:
def listToFile(aList,fileName):

    """ 
    RECIBE: Un nombre para el archivo que va a almacenar la lista y 
            una lista a almacenar
    """
    
    file = open(generateAbsolutePath(fileName),'w')
    
    for e in aList:
        file.write(str(e) + '&&&')
        
    file.close()    

In [42]:
def extractTextFrom(data):

    """ 
    RECIBE: El data frame de tweets
    TENER EN CUENTA: 
            Considera el campo location, keyword y text
            No altera al data frame
    DEVUELVE: En una lista como elementos la concatenacion de campo keyword, location y text
    """
    data_copy = data.copy()
    data_copy['keyword'].fillna('', inplace = True)
    data_copy['location'].fillna('', inplace = True)
    
    return list(data_copy['keyword'] + ' ' + data_copy['location'] + ' ' + data_copy['text'])

In [43]:
def mergeTextIn(row):
    
    """ 
    RECIBE: Una row de data frame de tweets
    TENER EN CUENTA: 
            Considera el campo location, keyword y text
            No altera al data frame
    DEVUELVE: La concatenacion de campo keyword, location y text
    """
    
    return row['keyword'] + ' ' + row['location'] + ' ' + row['text']

In [44]:
def extractNounFrequencies(data,nouns):
    
    """ 
    RECIBE: El data frame de tweets y una lista de nouns
    TENER EN CUENTA: 
            Considera el campo location, keyword y text
            No altera al data frame
    DEVUELVE: Un diccionario de frecuencia de nouns en el dataframe y otro de frecuencia en desastres
    """
    
    data_copy = data.copy()
    data_copy['keyword'].fillna('', inplace = True)
    data_copy['location'].fillna('', inplace = True)
    
    disasterNounFrequency = {}
    totalNounFrequency = {}
    
    for index, row in data_copy.iterrows():
    
        text = mergeTextIn(row)

        for word in nouns:

            if word in text:

                if word in totalNounFrequency:
                    totalNounFrequency[word] += text.count(word)

                    if row['target'] == 1:

                        if word in disasterNounFrequency:
                            disasterNounFrequency[word] += text.count(word)

                        else:
                            disasterNounFrequency[word] = text.count(word)

                else:
                    totalNounFrequency[word] = text.count(word)
                    
    return disasterNounFrequency,totalNounFrequency

In [45]:
def getWordIntrinsicNegativity(word,disasterNounFrequency,totalNounFrequency):
    
    """ 
    RECIBE: Una palabra y los mapas de frecuencia en desastre y total
    DEVUELVE: La frecuencia de uso de word en desastres / la frecuencia de word en todo el data frame
    """
    
    if word in disasterNounFrequency and word in totalNounFrequency:
        return disasterNounFrequency[word] / totalNounFrequency[word]
    else:
        return 0

In [46]:
def calculateNegativityMap(nouns,disasterNounFrequency,totalNounFrequency):
   
    """ 
    RECIBE: El data frame de tweets y los mapas de frecuencia en desastre y total
    DEVUELVE: Un diccionario que tiene como key nouns y value su negatividad
    """
    nounNegativityMap = {}
    
    for noun in nouns:
        
        nounNegativityMap[noun] = getWordIntrinsicNegativity(noun,disasterNounFrequency,totalNounFrequency)
        
    return nounNegativityMap

In [47]:
def isAsociated(word1,word2,contextualMap):
    
    """ 
    RECIBE: Dos palabras y un contexto
    DEVUELVE: Si la palabra 1 esta en el conjunto de palabras asociadas a la palabra 2
    """
    
    if not word2 in contextualMap:
        return False

    for pair in contextualMap[word2]:
        
        if pair[0] == word1:
            return True
    
    return False

In [48]:
def getContextualNegativity(word,words,context):
    
    """ 
    RECIBE: Una palabra, una lista de palabras y un contexto
    DEVUELVE: La negatividad contextual por la relacion entre estas palabras
    """
    
    contextualNegativity = 0
    
    for w in words:
        
        if isAsociated(w,word,context.englishContextualMap) and w != word:
            
            reinforcement = [asociate[1] for asociate in context.englishContextualMap[w] if asociate[0] == word][0]
            totalReinforcementMass = sum([asociate[1] for asociate in context.englishContextualMap[w]])
            
            contextualNegativity += reinforcement/totalReinforcementMass
            
    return contextualNegativity

In [49]:
def getTextNegativity(text,context):
    
    """ 
    RECIBE: Un texto y un context
    DEVUELVE: Un numero de 0 a 1 donde 1 es completamente negativo y 0 es nada negativo
    """
    
    words = text.split(' ')
    
    cleanedUpWords = []
    
    for word in words:
        
        if word != '':
            mostSimilar = getMaxSimilarityWord(word,context.nouns)
            
            if mostSimilar != '':
                cleanedUpWords.append(mostSimilar)
    
    intrinsicNegativity = 0
    contextualNegativity = 0
    totalIntrinsicNegativity = 0
    totalContextualNegativity = 0
    contextualPonderation = 0
    
    for word in cleanedUpWords: 
        
        intrinsicNegativity = context.nounNegativityMap[word]
        contextualNegativity =  getContextualNegativity(word,cleanedUpWords,context)
        
        if contextualNegativity > 0:
            contextualPonderation += 1
            
        totalIntrinsicNegativity += intrinsicNegativity
        totalContextualNegativity += contextualNegativity * intrinsicNegativity
    
    #print("TOTAL NEGATIVITY: {}, NEGATIVITY EXTRACTED FROM CONTEXT: {}".format(totalIntrinsicNegativity + totalContextualNegativity * contextualPonderation,totalContextualNegativity * contextualPonderation))
    return totalIntrinsicNegativity + totalContextualNegativity * contextualPonderation

In [50]:
class Context:
    
    """ 
    Clase que engloba toda las operaciones y nociones de contexto de un texto
    """
    
    def __init__(self,data = None, similarityThreshold = 0.8, load = ''):
        
        """ 
        RECIBE: 
            texts es una lista de textos (tweets) a entender
            similarityThreshold es una tolerancia para la similaridad. 
                                Si dos palabras superan ese limite se consideran simialres
            load es opcional y es el nombre del archivo que almacena
                                el contextualMap guardado en disco en caso de haberlo hecho
        """
        
        self.stopWords = set(stopwords.words('english'))
        self.similarityThreshold = similarityThreshold
        
        self.__atrbt_texts = load + '_' + 'texts'
        self.__atrbt_words = load + '_' + 'words'
        self.__atrbt_englishWords = load + '_' + 'englishWords'
        self.__atrbt_nonEnglishWords = load + '_' + 'nonEnglishWords'
        self.__atrbt_nouns = load + '_' + 'nouns'
        
        self.__atrbt_wordNet = load + '_' + 'wordNet'
        self.__atrbt_disasterNounFrequency = load + '_' + 'disasterNounFrequency'
        self.__atrbt_totalNounFrequency = load + '_' + 'totalNounFrequency'
        self.__atrbt_nounNegativityMap = load + '_' + 'nounNegativityMap'
        self.__atrbt_contextualMap = load + '_' + 'contextualMap'
        self.__atrbt_englishContextualMap = load + '_' + 'englishContextualMap'
            
            
        #====================================================
        self.data = data
        print("Data loaded")
        
        #====================================================
        if path.exists(self.__atrbt_texts):
            self.texts = readFromFileIntoList(self.__atrbt_texts)
            print("Texts loaded")
        else:
            self.texts = cleanUpWords(extractTextFrom(data))
            print("Extracted texts")
        
        #====================================================
        if path.exists(self.__atrbt_words):
            self.words = readFromFileIntoList(self.__atrbt_words, strings = True)
            print("Words loaded")
        else:
            self.words = mergeIntoWords(self.texts)
            print("Extracted words (without weird characters, no empty words and all to lower case)")
        
        #====================================================
        if path.exists(self.__atrbt_englishWords) and path.exists(self.__atrbt_nonEnglishWords):
            
            self.englishWords = readFromFileIntoList(self.__atrbt_englishWords, strings = True)
            print("English Words loaded")
            
            self.nonEnglishWords = readFromFileIntoList(self.__atrbt_nonEnglishWords, strings = True)
            print("Non English Words loaded")
        else:
            self.englishWords, self.nonEnglishWords = clasifyWordsRespectToEnglish(self.words)
            print("Words clasified into enlgish and non english")
        
        #====================================================
        if path.exists(self.__atrbt_nouns):
            self.nouns = readFromFileIntoList(self.__atrbt_nouns, strings = True)
            print("Nouns loaded")
        else:
            self.nouns = furtherCleanUp(self.englishWords,self.stopWords)
            print("Extracted nouns")
        
        #====================================================
        if path.exists(self.__atrbt_wordNet + '.csv'):
            self.wordNet = readDic(self.__atrbt_wordNet)
            print("Word Net loaded")
        else:
            self.wordNet = getContextualKnowledge(self.texts)
            print("Word Net produced")
        
        #====================================================
        if path.exists(self.__atrbt_disasterNounFrequency + '.csv') and path.exists(self.__atrbt_totalNounFrequency + '.csv'):
            
            self.disasterNounFrequency = readDic(self.__atrbt_nounNegativityMap)
            print("Disaster Frequency map loaded")
            
            self.totalNounFrequency = readDic(self.__atrbt_nounNegativityMap)
            print("Total Frequency map loaded")
        else:
            self.disasterNounFrequency, self.totalNounFrequency = extractNounFrequencies(self.data,self.nouns)
            print("Total and Disaster frequencies calculated for every noun")
        
        #====================================================
        if path.exists(self.__atrbt_nounNegativityMap + '.csv'):
            self.nounNegativityMap = readDic(self.__atrbt_nounNegativityMap)
            print("Noun Negativity map loaded")
        else:
            self.nounNegativityMap = calculateNegativityMap(self.nouns,self.disasterNounFrequency,self.totalNounFrequency)
            print("Negativity map produced")
        
        #====================================================
        if path.exists(self.__atrbt_contextualMap + '.csv'):
            self.contextualMap = readDic(self.__atrbt_contextualMap, transformStringedListToList = True)
            print("Contextual map loaded")
        else:
            self.contextualMap = getContextualMap(self.wordNet)
            print("Contextual map produced")
        
        #====================================================
        if path.exists(self.__atrbt_englishContextualMap + '.csv'):
            self.englishContextualMap = readDic(self.__atrbt_englishContextualMap, transformStringedListToList = True)  
            print("English Truncated Contextual map loaded")
        else:
            self.englishContextualMap = truncateMapTo(self.contextualMap,self.nouns)
            print("Enlgish truncated contextual map produced")
        
        
    def related(self, word, r = 1, restrict = True):
        
        """ 
        RECIBE: La palabra de la que se quiere conocer sus asociados y el orden minimo de su asociacion (su reinforcement)
                restric es para quedarse solo con las relaciones con palabras filtradas
        DEVUELVE: Una lista de todos los asociados (palabras, orden) que cumplen con el r
        """
        
        if restrict:
            asociated = self.englishContextualMap[getMaxSimilarityWord(word,self.words)]
        else:
            asociated = self.contextualMap[getMaxSimilarityWord(word,self.words)]
        
        if r == 1:
            return asociated
        
        strongAsociated = []
        
        for pair in asociated:
            if pair[1] >= r:
                strongAsociated.append(pair)
                                
        return strongAsociated
    
    
    def save(self,name):
        
        """ 
        RECIBE: El nombre de archivo que guarda el contextualMap
        DEVUELVE: Guarda el contextualMap en un csv
        """
        
        if name == '':
            return None
        
        #====================================================
        if not path.exists(self.__atrbt_texts):
            
            listToFile(self.texts, name + '_' + 'texts')
            print("Texts saved")
        
        #====================================================
        if not path.exists(self.__atrbt_words):
            
            listToFile(self.words, name + '_' + 'words') 
            print("Words saved")
        
        #====================================================
        if not path.exists(self.__atrbt_englishWords):
            
            listToFile(self.englishWords, name + '_' + 'englishWords')
            print("English Words saved")

        #====================================================
        if not path.exists(self.__atrbt_nonEnglishWords):
            
            listToFile(self.nonEnglishWords, name + '_' + 'nonEnglishWords')
            print("Non English Words saved")
        
        #====================================================
        if not path.exists(self.__atrbt_nouns):
            
            listToFile(self.nouns, name + '_' + 'nouns')
            print("Nouns saved")
        
        #====================================================
        if not path.exists(self.__atrbt_wordNet + '.csv'):
            
            dicToDataFrame(self.wordNet).to_csv(name + '_' + 'wordNet' + '.csv')
            print("Word Net saved")
        #====================================================
        if not path.exists(self.__atrbt_disasterNounFrequency + '.csv'):
            
            dicToDataFrame(self.disasterNounFrequency).to_csv(name + '_' + 'disasterNounFrequency' + '.csv')
            print("Disaster Frequency map saved")
        
        #====================================================
        if not path.exists(self.__atrbt_totalNounFrequency + '.csv'):
            
            dicToDataFrame(self.totalNounFrequency).to_csv(name + '_' + 'totalNounFrequency' + '.csv')
            print("Total Frequency map saved")
        
        #====================================================
        if not path.exists(self.__atrbt_nounNegativityMap + '.csv'):
            
            dicToDataFrame(self.nounNegativityMap).to_csv(name + '_' + 'nounNegativityMap' + '.csv')
            print("Noun Negativity map saved")
        
        #====================================================
        if not path.exists(self.__atrbt_contextualMap + '.csv'):
            
            dicToDataFrame(self.contextualMap).to_csv(name + '_' + 'contextualMap' + '.csv')
            print("Contextual map saved")
        
        #====================================================
        if not path.exists(self.__atrbt_englishContextualMap + '.csv'):
            
            dicToDataFrame(self.englishContextualMap).to_csv(name + '_' + 'englishContextualMap' + '.csv')  
            print("English Truncated Contextual map saved")
            
            

In [51]:
def probabilityOfGivenThat(eventA,eventB,data):
    
    """ 
    RECIBE: Un data frame y dos funciones booleanas que dado una row del data frame deciden corresponden
            o no a los eventos o condiciones A y B
    DEVUELVE: Devuelve la probabilidad condicional P(A|B) tomando como espacio muestral equiprobable el
              dataframe
    """
    
    frequencyOfAandB = data[ data.apply(lambda row : eventA(row) and eventB(row), axis = 1)]['id'].count()
    frequencyOfB = data[ data.apply(lambda row : eventB(row), axis = 1)]['id'].count()
    
    return frequencyOfAandB/frequencyOfB

In [52]:
def hasEnglishWord(text):
    """
    RECIBE: Un string.
    DEVUELVE: True si tiene al menos una palabra en ingles, sino retorna FALSE
    """
    englishDictionary = enchant.Dict("en_US")
    text = takeOutSpecialCaractersFromText(text)
    textSplitted = text.split()
    for word in textSplitted:
        if englishDictionary.check(word):
            return True
    return False

In [53]:
def englishPercentage(text):
    text = text.lower()
    text = takeOutSpecialCaractersFromText(text)
    englishDictionary = enchant.Dict("en_US")
    textSplitted = text.split()
    cantidadDePalabrasEnIngles = 0
    for word in textSplitted:
        if englishDictionary.check(word):
            cantidadDePalabrasEnIngles += 1
    return (cantidadDePalabrasEnIngles / len(textSplitted)) * 100